In [2]:
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2'
sys.path.insert(0,my_path + r'/code/COMMON')

# imports
import numpy as np 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Glove model

In [3]:
from dictionary_helpers import build_glove_dict

In [4]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/twitter_datasets_stanford/')

In [5]:
filename_glove_dict = 'glove.twitter.27B.25d.txt'

glove = build_glove_dict(filename_glove_dict)

# Load stop words

In [6]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/stop_words/')

In [7]:
filename_stopwords = 'stop_word_freq_min_100_ratio_marg_0.1.txt'

stop_words = []
with open(filename_stopwords, 'r', encoding='utf-8-sig') as f:
    for line in f:
        stop_words.append(line.lstrip().split()[0])
    del stop_words[-1]
    
print("File :", filename_stopwords)
print("Number of stop words :", len(stop_words))

File : stop_word_freq_min_100_ratio_marg_0.1.txt
Number of stop words : 630


# Load tfid 

In [8]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/tfidf/')

In [9]:
filename_tfidf = 'tfidf.txt'

tfidf = {}
with open(filename_tfidf, 'r', encoding='utf-8-sig') as f:
    next(f) # skip headers
    for line in f:
        word = line.strip().split()[0]
        tf = float(line.strip().split()[1])
        idf = float(line.strip().split()[2])
        tfidf[word] = [tf, idf]

# Build tweet vector method

In [10]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/code/tom/')

In [11]:
from tweet_processings import build_tweet_vector

In [12]:
# method to build tweet vector
method = ["mean"]

# Build tweet vectors TRAIN

In [13]:
import os
os.chdir(r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2\data\twitter_datasets_epfl\short')

In [14]:
# build positive tweet feature set
X_pos = []

with open('train_pos_processed.txt') as f:
    for line in f:
        tweet = line.lstrip().split()
        tweet_vector = build_tweet_vector(tweet, glove, tfidf, stop_words, method)
        if len(tweet_vector):
            X_pos.append(tweet_vector)
        
X_pos = np.array(X_pos)

'lisining to music' is an empty tweet.
'off to burgerboys' is an empty tweet.
'walmart with mommiiee' is an empty tweet.
'off to walmart' is an empty tweet.
'up with andy' is an empty tweet.
'with deanza' is an empty tweet.
'going to crowely' is an empty tweet.


In [15]:
# build negative tweet feature set
X_neg = []

with open('train_neg_processed.txt') as f:
    for line in f:
        tweet = line.lstrip().split()
        tweet_vector = build_tweet_vector(tweet, glove, tfidf, stop_words, method)
        if len(tweet_vector):
            X_neg.append(tweet_vector)
        
X_neg = np.array(X_neg) 

'enough' is an empty tweet.
'im thinking too much' is an empty tweet.
'kgnbgtini hugmepleaseprince ~' is an empty tweet.
'scoiled my tongue' is an empty tweet.
'so insanly busy' is an empty tweet.
'<number><number> philly' is an empty tweet.
'willy willy willy willy' is an empty tweet.


In [16]:
# labels
y_pos = np.ones(X_pos.shape[0])
y_neg = -np.ones(X_neg.shape[0])

In [17]:
# number of training samples
N_samples_train = -1

# cut samples
X_pos_cut = X_pos[:N_samples_train,:]
X_neg_cut = X_neg[:N_samples_train,:]

# cut targets
y_pos_cut = y_pos[:N_samples_train]
y_neg_cut = y_neg[:N_samples_train]

# concatenate
X_pos_neg = np.concatenate([X_pos_cut, X_neg_cut])
y_pos_neg = np.concatenate([y_pos_cut, y_neg_cut])

# Build tweet vectors TEST

In [18]:
import os
os.chdir(r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2\data\twitter_datasets_epfl\short')

In [19]:
X_test = []

with open('test_data_no_id_processed.txt') as f:
    for line in f:
        tweet = line.lstrip().split()
        tweet_vector = build_tweet_vector(tweet, glove, tfidf, stop_words, method)
        if len(tweet_vector):
            X_test.append(tweet_vector)
        
X_test = np.array(X_test) 
print(X_test.shape)

(10000, 25)


# Standardization

In [20]:
# set to "True" to standardize
ifStandardize = False

In [21]:
from sklearn.preprocessing import StandardScaler

if ifStandardize:
    
    scaler = StandardScaler()
    scaler.fit(X_pos_neg)
    X_pos_neg = scaler.transform(X_pos_neg)
    X_test = scaler.transform(X_test)

# KNN

In [33]:
from sklearn.neighbors import KNeighborsClassifier

# init
K = 100
clf = KNeighborsClassifier(n_neighbors=K)

# fit
clf.fit(X_pos_neg, y_pos_neg) 

# predict
y_pred = clf.predict(X_test)

# Submission

In [34]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/submissions/')

In [35]:
from create_csv_submission import create_csv_submission
import time
import datetime
i = datetime.datetime.now()

name = "sub_" + time.strftime("%d_%m_%Y") +  "_%sh_%smin" % (i.hour, i.minute)
#name = "vrf"
ids_test = range(1, X_test.shape[0]+1)
create_csv_submission(ids_test, y_pred, name)